In [3]:
import concurrent.futures

import polars as pl

from mrnarchitect.sequence import Sequence
from mrnarchitect.utils.analysis import hydrate

In [4]:
df = pl.read_csv("optimized-sequences.csv")
df

index,source,name,raw_input_sequence,input_sequence,error,optimized_sequence,optimization_error
i64,str,str,str,str,str,str,str
4721,"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000115947|ENSG0000011594…","""ATGAGCAGTCGTAAATCAAAGAGTAACAGC…","""ATGAGCAGTCGTAAATCAAAGAGTAACAGC…",null,"""ATGAGCAGCAGAAAGAGCAAGAGCAACAGC…",null
39774,"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000160953|ENSG0000016095…","""ATGGCGGATGCCAAGTATGTCCTCTGCCGA…","""ATGGCGGATGCCAAGTATGTCCTCTGCCGA…",null,"""ATGGCCGACGCCAAGTACGTGCTGTGCAGA…",null
27306,"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000112787|ENSG0000011278…","""ATGGAGGCCAAGGTCCGCCCGAGCCGGCGC…","""ATGGAGGCCAAGGTCCGCCCGAGCCGGCGC…",null,"""ATGGAGGCCAAGGTGAGACCCAGCAGAAGA…",null
27148,"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000198885|ENSG0000019888…","""ATGAATGTTGATGCAGAGGCCTCCATGGCT…","""ATGAATGTTGATGCAGAGGCCTCCATGGCT…",null,"""ATGAACGTGGACGCCGAGGCCAGCATGGCC…",null
20254,"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000204410|ENSG0000020441…","""ATGGCCTCCTTAGGAGCGAACCCAAGGAGG…","""ATGGCCTCCTTAGGAGCGAACCCAAGGAGG…",null,"""ATGGCCAGCCTCGGAGCCAACCCTAGAAGA…",null
…,…,…,…,…,…,…,…
50708,"""iedb_antigen_aa_seqs.fa""","""sp|Q8IX19|MCEM1_HUMAN Mast cel…","""MEVEEIYKHQEVKMQAPAFRDKKQGVSAKN…","""ATGGAGGTGGAGGAGATCTACAAGCACCAG…",null,"""ATGGAGGTGGAGGAGATCTACAAGCACCAG…",null
50709,"""iedb_antigen_aa_seqs.fa""","""sp|Q9D287|SPF27_MOUSE Pre-mRNA…","""MAGTGLVAGEVVVDALPYFDQGYEAPGVRE…","""ATGGCCGGCACCGGCCTGGTGGCCGGCGAG…",null,"""ATGGCCGGCACAGGCCTGGTGGCTGGCGAG…",null
50710,"""iedb_antigen_aa_seqs.fa""","""tr|Q4CN05|Q4CN05_TRYCC Trans-s…","""MSRHLFYSAVLLLLVVMCCGTAAVNAEELS…","""ATGAGCAGACACCTGTTCTACAGCGCCGTG…",null,"""ATGAGCAGACACCTGTTCTACAGCGCCGTG…",null


In [ ]:
def _hydrate(input_: dict) -> dict:
    input_sequence = input_["input_sequence"]
    hydrated_input = {}
    optimized_sequence = input_["optimized_sequence"]
    hydrated_optimized = {}
    if input_sequence:
        hydrated_input = hydrate(Sequence.create(input_sequence))
    if optimized_sequence:
        hydrated_optimized = hydrate(Sequence.create(optimized_sequence))
    
    return {
        **input_,
        **{f"input_{k}": v for k, v in hydrated_input.items()},
        **{f"optimized_{k}": v for k, v in hydrated_optimized.items()},
    }

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = list(executor.map(
        _hydrate,
        list(df.rows(named=True)),
    ))




In [9]:
results

<generator object _chain_from_iterable_of_lists at 0x7f4571cd6c20>

In [10]:
hydrated_df = pl.DataFrame(list(results))
hydrated_df.write_csv("output-hydrated.csv")
hydrated_df

shape: (0, 0)
┌┐
╞╡
└┘